In [56]:
file_path = './data/'

In [57]:
from collections import defaultdict
import unicodedata as ud
import pandas as pd
import numpy as np
import math
import ast
import re

In [58]:
%matplotlib inline
import matplotlib.pyplot as plt
from io import BytesIO
import pandas as pd
import numpy as np
import requests
import seaborn as sns
np.random.seed(0)
plt.style.use("ggplot")
import tensorflow as tf

In [59]:
# read data train
df_train = pd.read_csv(f'./data/VLSP2013_POS_train_BI_POS_Column.txt', sep='\t', encoding='utf-8', skip_blank_lines=False, header=None)
# Đổi tên các cột.
df_train.columns = ('Words','Tags')
# Đưa về dạng chữ thường.
df_train["Words"]=df_train["Words"].str.lower()

In [60]:
# read data test
df_test = pd.read_csv(f'./data/VLSP2013_POS_test_BI_POS_Column.txt', sep='\t', encoding='utf-8', skip_blank_lines=False, header=None)
# Đổi tên các cột.
df_test.columns = ('Words','Tags')
# Đưa về dạng chữ thường.
df_test["Words"]=df_train["Words"].str.lower()

In [61]:
# read data dev
df_dev = pd.read_csv(f'./data/VLSP2013_POS_dev_BI_POS_Column.txt', sep='\t', encoding='utf-8', skip_blank_lines=False, header=None)
# Đổi tên các cột.
df_dev.columns = ('Words','Tags')
# Đưa về dạng chữ thường.
df_dev["Words"]=df_dev["Words"].str.lower()

In [62]:
df_train[:20]

,Words,Tags
0,hải_tặc,N
1,eo_biển,N
2,malacca,Np
3,(,CH
4,kỳ,N
5,1,M
6,),CH
7,:,CH
8,eo_biển,N
9,không,R


In [63]:
# Đánh dấu stt câu theo vị trí của dấu chấm
def mark_sentences(tmp):
    sentences_index = [0]
    y = 0
    for i in range(len(tmp)):
        for _ in range(int(tmp[i]-y)):
            sentences_index.append(i)
        y = tmp[i]
    return sentences_index
    
    

In [64]:
# Tìm các vị trí trong tập train có tag = NaN => kết thúc câu
list_index = df_train[df_train['Tags'].isnull()].index
list_mark_sentences = mark_sentences(list_index)
df_train['Sentences'] = list_mark_sentences
df_train.drop(list_index,inplace=True)

In [65]:
# Tìm các vị trí trong tập train có tag = NaN => kết thúc câu
list_index = df_test[df_test['Tags'].isnull()].index
list_mark_sentences = mark_sentences(list_index)
df_test['Sentences'] = list_mark_sentences
df_test.drop(list_index,inplace=True)

In [66]:
# Tìm các vị trí trong tập train có tag = NaN => kết thúc câu
list_index = df_dev[df_dev['Tags'].isnull()].index
list_mark_sentences = mark_sentences(list_index)
df_dev['Sentences'] = list_mark_sentences
df_dev.drop(list_index,inplace=True)

In [67]:
# Kiểm tra số lượng nhãn.
print((df_dev['Tags'].nunique()))
print((df_test['Tags'].nunique()))
print((df_train['Tags'].nunique()))

20
21
32


In [68]:
df_train["Tags"].replace(',', 'CH', inplace=True)
# format nhãn B.
df_train['Tags'].replace(['Ab', 'Cb', 'Eb', 'Mb', 'Nb', 'Pb', 'Vb'],
                         ['B', 'B', 'B', 'B', 'B', 'B', 'B'], inplace=True)
# format nhãn Y.
df_train['Tags'].replace(['Ny', 'Vy', 'Xy'],
                         ['Y', 'Y', 'Y'], inplace=True)
# format nhãn B.
df_dev['Tags'].replace(['Ab', 'Cb', 'Eb', 'Mb', 'Nb', 'Pb', 'Vb'],
                       ['B', 'B', 'B', 'B', 'B', 'B', 'B'], inplace=True)
# format nhãn Y.
df_dev['Tags'].replace(['Ny', 'Vy', 'Xy'],
                       ['Y', 'Y', 'Y'], inplace=True)

# format nhãn B.
df_test['Tags'].replace(['Ab', 'Cb', 'Eb', 'Mb', 'Nb', 'Pb', 'Vb'],
                        ['B', 'B', 'B', 'B', 'B', 'B', 'B'], inplace=True)
# format nhãn Y.
df_test['Tags'].replace(['Ny', 'Vy', 'Xy', 'NY'],
                        ['Y', 'Y', 'Y', 'Y'], inplace=True)


In [69]:
print((df_dev['Tags'].nunique()))
print((df_test['Tags'].nunique()))
print((df_train['Tags'].nunique()))

19
20
21


In [70]:
df_train[df_train['Words'].isnull()]


,Words,Tags,Sentences
257911,NaN,N,11509
304459,NaN,N,13439
310763,NaN,N,13693
310930,NaN,N,13701


In [71]:
# Thay thế các word là từ nan bị đánh dấu là NaN
df_dev=df_dev.fillna('nan')
df_train=df_train.fillna('nan')

In [72]:
def get_sentences(df):
    arg_func = lambda s : [(w,t) for w, t in zip(s['Words'].values.tolist(), s['Tags'].values.tolist())]
    grouped = df.groupby('Sentences').apply(arg_func)
    print(grouped)
    sentences = [s for s in grouped]
    return sentences

In [ ]:
sentences_train = get_sentences(df_train)
sentences_test = get_sentences(df_test)
sentences_dev = get_sentences(df_dev)

In [74]:
print(sentences_test[1])

[('lưu_thông', 'V'), ('đường_biển', 'N'), ('của', 'N'), ('thế_giới', 'R'), (',', 'V'), ('đó', 'M'), ('là', 'N'), ('hải_trình', 'A'), ('lớn', 'A'), ('nhất', 'E'), ('từ', 'P'), ('tây', 'L'), ('sang', 'N'), ('đông', 'A'), ('với', 'E'), ('50.000', 'N'), ('lượt', 'A'), ('tàu_bè', 'CH')]


In [75]:
for sentence in sentences_dev:
    sentences_train.append(sentence)

In [124]:
# Tạo ra file mới với words đã được gán tags từ tập train và test

with open('./data/test_words_tags.txt','w', encoding='utf-8') as f:
    for sentence in sentences_test:
        for z in sentence:
            f.write(f'{z[0]}\t{z[1]}\n')
        f.write(f'\n')
with open('./data/train_words_tags.txt','w', encoding='utf-8') as f:
    for sentence in sentences_train:
        for z in sentence:
            f.write(f'{z[0]}\t{z[1]}\n')
        f.write(f'\n')

In [77]:
# Tạo ra file mới với chỉ có words từ tập train và test

with open('./data/test_words.txt','w', encoding='utf-8') as f:
    for sentence in sentences_test:
        for z in sentence:
            f.write(f'{z[0]}\n')
        f.write(f'\n')
with open('./data/train_words.txt','w', encoding='utf-8') as f:
    for sentence in sentences_train:
        for z in sentence:
            f.write(f'{z[0]}\n')
        f.write(f'\n')

In [95]:
# Xây dựng tập từ vựng
words_1 = tuple(df_train['Words'].values)
words_2 = tuple(df_test['Words'].values)
words = [x for x in set(words_1+words_2)]


In [91]:
vocabs = {w: i+2 for i, w in enumerate(words)}
vocabs['--n--'] = 0
vocabs['--unk--'] = 1


In [96]:
print(len(vocabs))
count = 0
for key, value in vocabs.items():
    print(key, value)
    count += 1
    if count > 20: break
    

22505
mắt 2
khò 3
mbc 4
lừa_đảo 5
phó_viện_trưởng 6
mỏi_mòn 7
xé_rào 8
csiro 9
đường_sá 10
lâm_nghiệp 11
dẹt 12
hiệp_hội 13
đồng_minh 14
trao_tặng 15
35 16
nguyễn_bá_thụ 17
ampli 18
ngoại_tỉnh 19
tôt 20
trung_bình 21
nhúng 22


In [120]:
# Hàm xử lý các từ chưa có trong vocabs (chưa có tags)

def preprocess(vocabs, path):
    data = []
    file = open(path, encoding='utf-8').readlines()
    for word in file:
        if not word.split(): # Nếu là khoảng trắng sẽ là --n--
            word = '--n--'
            data.append(word)
            continue
        elif word.strip() not in vocabs: # Nếu từ đó không có trong vocab sẽ là --unk--
            word = '--unk--'
            data.append(word)
            continue
        data.append(word.strip())
    return data

In [98]:
def plot_tag_counts(words_tags):
    tags = [word_tag.split()[1] for word_tag in words_tags if word_tag.split()]
    tag_counts = pd.DataFrame(tags)[0].value_counts()
    tag_counts.plot.bar(rot=0, width=0.7, legend=False, figsize=(15, 5))
    return pd.DataFrame(tag_counts).T.assign(Total=tag_counts.sum())

In [107]:
train_words_tags = open('./data/train_words_tags.txt', encoding='utf-8').readlines()
print('Số lượng từ trong tập train_gold:', len(train_words_tags))
train_words_tags[0:5]

Số lượng từ trong tập train_gold: 659653


['hải_tặc\tN\n', 'eo_biển\tN\n', 'malacca\tNp\n', '(\tCH\n', 'kỳ\tN\n']

In [121]:
train_words = preprocess(vocabs, './data/train_words.txt')
print('Số lượng từ trong tập train_words:', len(train_words))

Số lượng từ trong tập train_words: 659653


In [116]:
# Các từ không nằm trong vocabs
arr = []
for word_tag, word in zip(train_words_tags, train_words):
    if word == '--unk--':
        arr.append(word_tag.split()[0])

In [123]:
test_words = preprocess(vocabs, './data/test_words.txt')
print('Số lượng từ trong tập test_words:', len(test_words))
test_words[0:5]

Số lượng từ trong tập test_words: 68466


['hải_tặc', 'eo_biển', 'malacca', '(', 'kỳ']

In [ ]:
test_words_tags = open('./data/test_words_tags.txt', encoding='utf-8').readlines()
print('Các từ không nằm trong vocabs', end=': ')
for word_tag, word in zip(test_words_tags, test_words):
    if word == '--unk--': print(word_tag.split()[0], end=', ')
plot_tag_counts(test_words_tags)

Phần 3: Gán nhãn

In [126]:
def seperate_word_tags(word_tag, vocabs):  # Gán nhãn cho các từ
    if not word_tag.split():
        word = '--n--'
        tag = '--s--'
    else:
        word, tag = word_tag.split()
        if word not in vocabs:
            word = '--unk--'
    return word, tag


def create_dictionary(train_words_tags, vocabs):
    emission_count = defaultdict(int)
    transition_count = defaultdict(int)
    tag_count = defaultdict(int)

    pre_tag = '--s--'

    for word_tag in train_words_tags:
        word, tag = seperate_word_tags(word_tag, vocabs)
        transition_count[(pre_tag, tag)] += 1
        emission_count[(tag, word)] += 1
        tag_count[tag] += 1
        pre_tag = tag
    return transition_count, emission_count, tag_count


In [130]:
transition_count, emission_count, tag_count = create_dictionary(train_words_tags, vocabs)
states = sorted(tag_count.keys())
print('Số nhãn:', len(states))
print(states)

Số nhãn: 22
['--s--', 'A', 'B', 'C', 'CH', 'Cc', 'E', 'I', 'L', 'M', 'N', 'Nc', 'Ni', 'Np', 'Nu', 'P', 'R', 'T', 'V', 'X', 'Y', 'Z']


In [ ]:
print("Transition examples: ")
for example in transition_count.items():
    print(example)

In [133]:
def predict_pos(words, words_tags, emission_count, vocabs, states):
    num_correct = 0
    for word, word_tag in zip(words, words_tags):
        word_tag = word_tag.split()
        if len(word_tag) !=2:
            continue
        else:
            true_label = word_tag[1]
        
        max_count = 0
        tag_final = ''
        
        if word not in vocabs:
            continue
        for tag in states:
            if(tag,word) not in emission_count: 
                continue
            count = emission_count[(tag, word)]
            if count > max_count:
                max_count = count
                tag_final = tag
        if tag_final == true_label:
            num_correct +=1
    accuracy = num_correct/len(words_tags)
    return accuracy
                

In [134]:
accuracy = predict_pos(train_words, train_words_tags, emission_count, vocabs, states)
print('Độ chính xác trên tập train:', accuracy)

Độ chính xác trên tập train: 0.8801582043892774
